In [1]:
import pandas as pd

In [52]:
data_tweets = pd.read_csv("original_twitts.csv", header = None)
data_retweets = pd.read_csv("original_retwitts.csv", header = None)

In [53]:
data_retweets = data_retweets.rename(columns={0:"user_id", 1:"time", 2:"retweet_id" ,3:"orig_id"})
data_tweets = data_tweets.rename(columns={0:"orig_id", 1:"user_id", 2:"time" ,3:"num"})

retweets = pd.DataFrame()
retweets["user_id"] = data_retweets.user_id
retweets["tweet_id"] = data_retweets.orig_id
retweets["time"] = data_retweets.time

tweets = pd.DataFrame()
tweets["user_id"] = data_tweets.user_id
tweets["tweet_id"] = data_tweets.orig_id
tweets["time"] = data_tweets.time

In [54]:
retweets.user_id = retweets.user_id.astype(int)
retweets.tweet_id = retweets.tweet_id.astype(int)
tweets.user_id = tweets.user_id.astype(int)
tweets.tweet_id = tweets.tweet_id.astype(int)

In [55]:
def just_h(x):
    return x.split(":", 1)[0]
retweets.time = retweets.time.apply(just_h)
tweets.time = tweets.time.apply(just_h)
retweets.time = pd.to_datetime(retweets.time, format='%Y-%m-%d-%H')
tweets.time = pd.to_datetime(tweets.time, format='%Y-%m-%d-%H')

In [66]:
mapping = open("uidlist.txt", "r") 
map_id = mapping.readlines()

map_list = []
for line in map_id:
    map_list.append(int(line.split()[0]))

map_list_reverse = {}
j = 0
for i in map_list:
    map_list_reverse[i] = j
    j = j + 1

tweets["num"] = data_tweets.num
tweets.num = tweets.num.astype(int)
tweets = tweets.sort_values(by = ["num"], ascending=False)
small_tweets = tweets[30:33]

In [67]:
small_tweets_ids = small_tweets.tweet_id.unique()

In [68]:
small_retweets = retweets[retweets["tweet_id"].isin(small_tweets_ids)]

In [69]:
unique_users_small = list(small_tweets.user_id.unique()) + list(small_retweets.user_id.unique())

In [70]:
len(unique_users_small)

56393

In [71]:
u_u_small_maped = []
for i in unique_users_small:
        u_u_small_maped.append(map_list_reverse[i])

In [72]:
file_2 = open("weibo_network.txt", "r") 
lines_2 = file_2.readlines()

In [73]:
inv_graph = {}
for i in u_u_small_maped:
    inv_graph[i] = []
    adj = lines_2[i+1].split()
    l = int(adj[1])
    j = 2
    while (j < (2*l + 2)):
        inv_graph[i].append(int(adj[j]))
        j = j + 2

In [74]:
set_unique_users = set(u_u_small_maped)
for key in inv_graph:
    inv_graph[key] = list(set(inv_graph[key]) & set_unique_users)

In [75]:
graph = {}
for key in inv_graph:
    for i in inv_graph[key]:
        if i in graph:
            graph[i].append(key)
        else:
            graph[i] = []
            graph[i].append(key)

In [76]:
maped_graph = {}
for key in graph:
    maped_graph[map_list[key]] = []
    for i in graph[key]:
         maped_graph[map_list[key]].append(map_list[i])

In [84]:
import csv
w = csv.writer(open("data/graph.csv", "w"))
for key, val in maped_graph.items():
    w.writerow([key, val])

In [83]:
small_retweets.to_csv("data/small_retweets.csv", sep='\t')
small_tweets.to_csv("data/small_tweets.csv", sep='\t')

In [79]:
tweets_ids = small_tweets.tweet_id

In [80]:
tweets_collection = {} 
for tweet in tweets_ids:
    tweets_collection[tweet] = small_retweets[small_retweets.tweet_id == tweet]

In [82]:
for key in tweets_collection:
        filename = "data/twitter_data/"+str(key)+".csv"
        tweets_collection[key].to_csv(filename, sep='\t')